In [8]:
import requests
from datetime import datetime, timedelta
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import certifi

In [9]:
def get_gps_sppo(data_inicial, data_final):
    url = "https://dados.mobilidade.rio/gps/sppo"
    params = {
        "dataInicial": data_inicial,
        "dataFinal": data_final
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

In [10]:
agora = datetime.now()
data_final = agora.strftime("%Y-%m-%d %H:%M:%S")
data_inicial = (agora - timedelta(seconds=10)).strftime("%Y-%m-%d %H:%M:%S")

try:
    dados_gps = get_gps_sppo(data_inicial, data_final)
    print(f"Total de registros recuperados: {len(dados_gps)}")
    if dados_gps:
        # Exibe o primeiro registro para conferência
        print(dados_gps[0])
except Exception as e:
    print(f"Erro ao consultar a API: {e}")

Total de registros recuperados: 117
{'ordem': 'D12123', 'latitude': '-22,8894', 'longitude': '-43,58953', 'datahora': '1767740930000', 'velocidade': '24', 'linha': '2308', 'datahoraenvio': '1767740931000', 'datahoraservidor': '1767740937000'}


import redis
import json

redis_client = redis.Redis(
    host="localhost",
    port=6379,
    decode_responses=True
)


In [ ]:
179.218.19.228

In [ ]:
import redis
import os

REDIS_URL = os.environ.get("REDIS_URL")

redis_client = redis.from_url(
    decode_responses=True
)


In [11]:
import json

TTL_ONIBUS = 120  # segundos

def salvar_onibus(dados):
    for d in dados:
        chave = f"onibus:{d['ordem']}"

        redis_client.setex(
            chave,
            TTL_ONIBUS,
            json.dumps(d)
        )


In [12]:
def listar_onibus_ativos():
    chaves = redis_client.keys("onibus:*")
    return [
        json.loads(redis_client.get(chave))
        for chave in chaves
    ]


In [ ]:
from datetime import datetime, timedelta
import time
from pymongo import UpdateOne

def coletar_e_salvar():
    agora = datetime.now()
    data_final = agora.strftime("%Y-%m-%d %H:%M:%S")
    data_inicial = (agora - timedelta(seconds=10)).strftime("%Y-%m-%d %H:%M:%S")

    try:
        dados_gps = get_gps_sppo(data_inicial, data_final)
        print(f"[{agora}] Registros recuperados: {len(dados_gps)}")

        if not dados_gps:
            return

        operacoes = []

        for registro in dados_gps:
            if "ordem" not in registro:
                continue

            operacoes.append(
                UpdateOne(
                    {"ordem": registro["ordem"]},  # filtro
                    {"$set": registro},             # substitui/atualiza
                    upsert=True                     # cria se não existir
                )
            )

        if operacoes:
            resultado = colecao.bulk_write(operacoes)
            print(
                f"Upserts: {resultado.upserted_count} | "
                f"Modificados: {resultado.modified_count}"
            )

    except Exception as e:
        print(f"Erro ao consultar ou gravar dados: {e}")
